<a href="https://colab.research.google.com/github/ancka019/ComputationsMethods6sem/blob/main/method2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
import pandas as pd
from math import sqrt
from numpy.linalg import inv, det, norm, cond
from scipy.linalg import hilbert

Mетод квадратного корня (Холецкого)

In [30]:
def Cholesky(A):
    L = np.zeros((A.shape[0], A.shape[0]))
    for i in range(A.shape[0]):
        for j in range(i):
            temp = 0
            for k in range(j):
                temp += L[i][k] * L[j][k]
            L[i][j] = (A[i][j] - temp) / L[j][j]
        temp = A[i][i]
        for k in range(i):
            temp -= L[i][k] * L[i][k]
        L[i][i] = sqrt(temp)
    return L    

Aлгоритм LU-декомпозиции

In [3]:
def lu(a):
    n = a.shape[0]
    l = np.identity(n)
    u = np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            if i <= j:
                u[i,j] = a[i,j]-sum([l[i,k]*u[k,j] for k in range(i)])
            else:
                l[i,j] = (a[i,j]-sum([l[i,k]*u[k,j] for k in range(j)]))/u[j,j]
    return l,u

Pешение СЛАУ LU-методом

In [4]:
def lu_solve(l,u,b=None): #решение СЛАУ LU-методом
    if b is None:
        b = np.random.uniform(-100,100,size=(u.shape[1]))
    y = np.zeros(l.shape[1])
    x = np.zeros(u.shape[1])
    n = len(x)
    for i in range(len(y)):
        y[i] = b[i] - sum([l[i,p]*y[p] for p in range(i)])
    for j in range(len(x)):
        x[n-j-1]=(y[n-j-1]-sum([u[n-j-1,n-p-1]*x[n-p-1] for p in range(j)]))/u[n-j-1,n-j-1]
    return x
            

In [5]:
def solve(L, b):
    y = np.linalg.solve(L, b)
    x = np.linalg.solve(L.T, y)
    return x

In [32]:
for size in [3, 7, 11]:
    A = hilbert(size)
    x = np.ones(size)
    b = np.dot(A, x)
    
    table = []
    print("\nМатрица Гильберта ", size, '*', size)
    for alpha in range(-12, 0):
        var_A = A + (10**alpha)*np.eye(size)

        table.append(
            {'α': 10**alpha,
            'cond(A)': cond(A),
            'cond(A+αE)': cond(var_A),
            '|x-x*|':  norm(x - solve(Cholesky(var_A), b))
            }
        )
        
        
    table = pd.DataFrame(table)
    print(table)

    x = np.random.uniform(0, 10, size=A.shape[0])
    b = np.dot(A, x)
    print('\nНаилучшее значение α:', table[table['|x-x*|'] == min(table['|x-x*|'])]['α'].array[0])
    
    print('Случайный вектор x:', x)
    diff_table = [
        ['Ax = b', norm(x - solve(Cholesky(A), b))],
        ['A + α * x = b', norm(x - solve(Cholesky(A + (10 ** alpha) * np.eye(A.shape[0])), b))],
        ['A + 10 * α * x = b', norm(x - solve(Cholesky(A + 10 * (10 ** alpha) * np.eye(A.shape[0])), b))],
        ['A + 0.1 * α * x = b', norm(x - solve(Cholesky(A + 0.1 * (10 ** alpha) * np.eye(A.shape[0])), b))]
    ]
    
    print(pd.DataFrame(diff_table, columns=['Уравнение', '|x-x*|']))


Матрица Гильберта  3 * 3
               α     cond(A)  cond(A+αE)        |x-x*|
0   1.000000e-12  524.056778  524.056777  3.855113e-11
1   1.000000e-11  524.056778  524.056776  3.853672e-10
2   1.000000e-10  524.056778  524.056758  3.853580e-09
3   1.000000e-09  524.056778  524.056583  3.853569e-08
4   1.000000e-08  524.056778  524.054831  3.853556e-07
5   1.000000e-07  524.056778  524.037315  3.853429e-06
6   1.000000e-06  524.056778  523.862213  3.852163e-05
7   1.000000e-05  524.056778  522.117620  3.839546e-04
8   1.000000e-04  524.056778  505.291334  3.717896e-03
9   1.000000e-03  524.056778  382.204731  2.830854e-02
10  1.000000e-02  524.056778  111.790091  9.450946e-02
11  1.000000e-01  524.056778   14.688460  3.188015e-01

Наилучшее значение α: 1e-12
Случайный вектор x: [8.95734608 5.89059218 4.28804588]
             Уравнение        |x-x*|
0               Ax = b  2.152399e-13
1        A + α * x = b  8.896364e-01
2   A + 10 * α * x = b  4.858649e+00
3  A + 0.1 * α * x = b  1.3